In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

In [2]:
gpus = tf.config.list_physical_devices("GPU")

I0000 00:00:1729880681.832464   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729880681.893588   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729880681.893624   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
path_to_data = os.getcwd() + '/dataset'

In [5]:
data = tf.keras.utils.image_dataset_from_directory(
    path_to_data,
    image_size=(128,128),  
    batch_size=32,        
    label_mode='int'     
)

Found 17034 files belonging to 6 classes.


I0000 00:00:1729880682.274309   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729880682.274435   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729880682.274490   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729880682.379560   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729880682.379605   75603 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [6]:
import os
os.listdir('.')

['.ipynb_checkpoints',
 'Untitled.ipynb',
 'dataset',
 'data_for_predictions',
 'logs']

In [7]:
os.getcwd()

'/home/sebastian/MachineLearning/IntelImageClassifier'

In [8]:
import glob

images = []

for dirr in ['0', '1', '2', '3', '4', '5']:
    images.extend(glob.glob(os.path.join(path_to_data, str(dirr), '*.jpg')))
len(images)

17034

In [9]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels = 3)
    
    image = tf.image.resize(image, [128,128])
    image = image /255.0
    
    return image

In [10]:
path_ds = tf.data.Dataset.from_tensor_slices(images)

image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

batch_size = 32
test_data = image_ds.batch(batch_size)

test_data = test_data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [11]:
def preprocess(image, label):
    image = image / 255.0
    label = tf.keras.utils.to_categorical(label, num_classes=6)
    return image, label

data = data.map(preprocess)

In [12]:
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()
batch[0][0]

array([[[0.2616345 , 0.29325047, 0.17949602],
        [0.4363157 , 0.42087498, 0.2900795 ],
        [0.52499974, 0.44762394, 0.28921545],
        ...,
        [0.6766798 , 0.6350828 , 0.5222067 ],
        [0.67969805, 0.6387209 , 0.5046729 ],
        [0.59702265, 0.557104  , 0.41564846]],

       [[0.27242696, 0.2870414 , 0.18878773],
        [0.34498483, 0.3289941 , 0.2116077 ],
        [0.46508405, 0.413073  , 0.2684039 ],
        ...,
        [0.6812761 , 0.6372779 , 0.5384586 ],
        [0.69699323, 0.6450018 , 0.5145951 ],
        [0.6626125 , 0.6094111 , 0.45964116]],

       [[0.23428835, 0.2303194 , 0.14676322],
        [0.30606353, 0.29696977, 0.19787478],
        [0.39032558, 0.36624467, 0.24795114],
        ...,
        [0.6797538 , 0.63496214, 0.5611601 ],
        [0.69946814, 0.6379289 , 0.5232285 ],
        [0.6633605 , 0.59176123, 0.44857106]],

       ...,

       [[0.42892012, 0.1291736 , 0.07067009],
        [0.41465873, 0.12270532, 0.06001982],
        [0.51236403, 0

In [13]:
len(data)

533

In [14]:
train_size = int(len(data) * .7)
test_size = int(len(data) * .1)
val_size = int(len(data) * .2)

In [15]:
train_data = data.take(train_size)
val_data = data.skip(train_size).take(val_size)
test_data = data.skip(train_size).skip(val_size).take(test_size)

In [16]:
len(train_data)

373

In [17]:
len(test_data)

53

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.regularizers import l2

In [19]:
model = Sequential()
model.add(Input(shape = (128, 128, 3)))
model.add(Conv2D(128,(3,3), activation = 'relu', input_shape = (128,128,3), kernel_regularizer = (l2(0.001))))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(256,(3,3), activation = 'relu', kernel_regularizer = (l2(0.001))))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(256, activation = 'relu', kernel_regularizer = (l2(0.001))))
model.add(Dropout(0.5))
model.add(Dense(6, activation = 'softmax'))

/home/sebastian/miniconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile('adam',loss=tf.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 230400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    58,982,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,282,950 (226.15 MB)

 Trainable params: 59,282,950 (226.15 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
hist = model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[tensorboard_callback, early_stopping])

Epoch 1/10


I0000 00:00:1729880684.005087   75790 service.cc:146] XLA service 0x7fe4940049c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729880684.005221   75790 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 Laptop GPU, Compute Capability 8.9


  3/373 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.1597 - loss: 7.2869

I0000 00:00:1729880687.227906   75790 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


373/373 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.4616 - loss: 2.1048 - val_accuracy: 0.6807 - val_loss: 1.0488
Epoch 2/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.6350 - loss: 1.1370 - val_accuracy: 0.7073 - val_loss: 0.9864
Epoch 3/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.6777 - loss: 1.0805 - val_accuracy: 0.7176 - val_loss: 0.9958
Epoch 4/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.7191 - loss: 0.9959 - val_accuracy: 0.7335 - val_loss: 0.9679
Epoch 5/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.7255 - loss: 0.9827 - val_accuracy: 0.7391 - val_loss: 0.9381
Epoch 6/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.7511 - loss: 0.9257 - val_accuracy: 0.7883 - val_loss: 0.8504
Epoch 7/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.7643 - loss: 0.8944 - val_accuracy: 0.7659 - val_loss: 0.8946
Epoch 8/10
373/373 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.7770 - loss: 0.8770 - val_accurac

In [23]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [24]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [25]:
predictions = []

for batch in test_data.as_numpy_iterator():
    X = batch 
    yhat = model.predict(X)
    predictions.append(yhat)

predictions = np.concatenate(predictions, axis=0)

predictions = np.array(predictions)

predicted_classes = np.argmax(predictions, axis=1)

print(predicted_classes)

print(f"Predictions shape: {predictions.shape}")

print(predictions[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━

In [26]:
import cv2

In [28]:
def load_and_preprocess_image_cv2(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    return img

def predict_class_cv2(image_path):
    image = load_and_preprocess_image_cv2(image_path)
    
    image = np.expand_dims(image, axis=0)
    
    prediction = model.predict(image)
    
    predicted_class = np.argmax(prediction, axis=1)[0]
    
    return predicted_class

new_image_path = os.getcwd() + '/data_for_predictions/seg_pred/seg_pred/6.jpg' 

predicted_class = predict_class_cv2(new_image_path)

if predicted_class == 0:
    print("This is a street")
elif predicted_class ==1:
    print("This is a sea")
elif predicted_class ==2:
    print("This is a mountain")
elif predicted_class ==3:
    print("This is a glacier")
elif predicted_class ==4:
    print("This is a forest")
elif predicted_class ==5:
    print("This is a building")    
    
print(f"The predicted class is: {predicted_class}")

image = load_and_preprocess_image_cv2(new_image_path)
plt.imshow(image)
plt.title(f"Predicted: {predicted_class}")
plt.axis("off")
plt.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
This is a sea
The predicted class is: 1
